In [1]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 1.2 MB/s eta 0:00:00


In [2]:
import pathlib
import google.generativeai as genai

In [3]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [5]:
model = genai.GenerativeModel('gemini-1.0-pro-001')

In [30]:
import os
import pandas as pd
from tqdm import tqdm
import time

def getGemini10Pro001Response(prompt):
    # sleep to avoid api limits, should take ~15 minutes in total
    # time.sleep(1)
    response = model.generate_content(prompt).text
    return response

def getLLMResponse(prompt):
    return getGemini10Pro001Response(prompt)

def getGeminiChatResponse(model,question,subquestions): #model type / large question in str/ subquestions in array

    header_instruction = ''' This is an example scenario and the rules defined for that scenario ''' ##### the one for starting the chat
    header_last_instruction=''' Given this, wait for the subproblems I give you and answer them accordingly   '''
    final_instruction = ''' Given the answers you have generated for the subproblems, answer the final question ''' ### what's the best answer etc...
    chat = model.start_chat(history=[])
    response = chat.send_message(header_instruction + question +header_last_instruction )
    for subs in subquestions:
        response = chat.send_message(subs)
    response  = chat.send_message(final_instruction)
    # return response.text
    history =""
    for message in chat.history:
        history += message.role + ": " + message.parts[0].text + "\n"
    return history, response

In [33]:
his,res = getGeminiChatResponse(model,"i will gave you some information first, and then ask you a question",["i'm 7,0","i'm 280lb","i can run 100m in 10s","i can shoot 3 point 100 in a row"])

In [34]:
print(his)

user:  i will gave you some information first, and then ask you a question
model: Sure. I'm ready for your information and question.
user: i'm 7,0
model: What is your question?
user: i'm 280lb
model: What is your question?
user: i can run 100m in 10s
model: What is your question?
user: i can shoot 3 point 100 in a row
model: What is your question?
user:  list all my advantage on basketball
model: * **Height:** At 7'0", you have a significant height advantage over most other players on the court. This gives you an advantage in rebounding, shot blocking, and defending the post.
* **Weight:** At 280 lbs, you have a significant weight advantage over most other players on the court. This gives you an advantage in strength and physicality, which can be helpful in rebounding, setting screens, and defending the post.
* **Speed:** You can run 100m in 10 seconds, which is very fast for a player of your size. This gives you an advantage in getting down the court on fast breaks and closing out on 

In [27]:
print(res)

* **Height:** At 7'0", you have a significant height advantage over most other players on the court. This gives you an advantage in rebounding, blocking shots, and scoring in the paint.
* **Weight:** At 280 lbs., you have a significant weight advantage over most other players on the court. This gives you an advantage in strength and physicality, which can be helpful in rebounding, setting screens, and defending in the post.
* **Speed:** Despite your size, you are able to run 100 meters in 10 seconds. This gives you an advantage in getting down the court quickly on offense and defense.
* **Shooting:** You are able to shoot 3-pointers with a high degree of accuracy. This gives you an advantage in spacing the floor and creating scoring opportunities for yourself and your teammates.

Overall, your physical attributes give you a number of advantages on the basketball court. You should use these advantages to your advantage and work to develop your skills in other areas of the game, such as 

In [24]:
# def getBaseAnswers(dataDf):
#     BASE_ANS_DIR = './Gemini_results.json'

#     saveDf = None#pd.DataFrame(columns=['question', 'reasoning', 'subproblems', 'label'])
#     #saveDf = #saveDf.astype(str)

#     for index, row in tqdm(dataDf.iterrows(), total=len(dataDf)):
#         prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
#         response = getLLMResponse(prompt)
#         # if saveDf is None:
#         #     saveDf = pd.DataFrame(columns=['question', 'reasoning', 'subproblems', 'label'])
#         #     saveDf = saveDf.astype(str)
#         # else:
#         #     saveDf = pd.read_json(BASE_ANS_DIR, dtype=str)

#         # new_record = {
#         #     'question': row['example'],
#         #     'gold-reasoning': row['proof'],
#         #     'subproblems': '',
#         #     'chat-gpt-3.5-turbo-ans': response,
#         #     'label': row['label'],
#         # }
#         # temp_df = pd.DataFrame([new_record])
#         # saveDf = pd.concat([saveDf, temp_df], ignore_index=True)
#         # saveDf.to_json(BASE_ANS_DIR, orient='records')
import csv
import json
def getBaseAnswers(dataDf):
    # Specify the path for the output CSV file
    output_csv_path = '/content/drive/MyDrive/gemini/data/Gemini_results.csv'

    # Open the CSV file for writing
    with open(output_csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write the header row
        writer.writerow(['Example', 'Response'])
        count = 0
        for index, row in tqdm(dataDf.iterrows(), total=dataDf.shape[0]):
            # Construct the prompt from the 'example' column
            prompt = row['example'] + "\n The label is what (proved, disproved, unknown)?"
            # Get the response from your API call function
            response = getLLMResponse(prompt)
            # Write the current 'example' and its 'response' to the CSV
            writer.writerow([row['example'], response])
            count += 1
            print("The response for " + str(count) + "is: ",response)

In [ ]:
def main():
    dataPath = "/content/drive/MyDrive/gemini/data/test.json"
    df = pd.read_json(dataPath, dtype=str)
    getBaseAnswers(df)

if __name__ == "__main__":
    main()

    # Load data
    # get initial response
    # break down questions using reasoning (custom model, LLAMA)
# Iteratively answer the subquestions based on the model, context, and subquestions
# get a final answer
# compare

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
dataPath = "/content/drive/MyDrive/gemini/data/output-subproblems-parsed.json"
dataDf = pd.read_json(dataPath, dtype=str)
for index, row in tqdm(dataDf.iterrows(), total=dataDf.shape[0]):
            # Construct the prompt from the 'example' column
            # Extract the "subproblems" attribute and split it into an array
            subproblems = row["subproblems"].split("||")
            for subproblem in subproblems:
                # Do something with each subproblem
                print(subproblem)
            # prompt = row['example']
             # Get the response from your API call function
            # response = getLLMResponse(prompt)
